In [1]:
#CAPSTONE PROJECT; LENDING IN ZAMBIA

In [88]:
import pandas as pd 
from faker import Faker
import random 
import matplotlib.pyplot as plt 
import seaborn as sns

In [89]:
!pip install faker

In [166]:
btc_data = pd.read_csv(r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\btc-usd-max.csv")
print(btc_data.head(20))

               bitcoin_date    price  bitcoin_sats  one_dollar_sats
0   2017-08-02 00:00:00 UTC  2697.85   100000000.0            37067
1   2017-08-03 00:00:00 UTC  2777.37   100000000.0            36005
2   2017-08-04 00:00:00 UTC  2845.90   100000000.0            35138
3   2017-08-05 00:00:00 UTC  3237.07   100000000.0            30892
4   2017-08-06 00:00:00 UTC  3189.73   100000000.0            31351
5   2017-08-07 00:00:00 UTC  3356.71   100000000.0            29791
6   2017-08-08 00:00:00 UTC  3410.45   100000000.0            29322
7   2017-08-09 00:00:00 UTC  3330.90   100000000.0            30022
8   2017-08-10 00:00:00 UTC  3367.91   100000000.0            29692
9   2017-08-11 00:00:00 UTC  3562.59   100000000.0            28069
10  2017-08-12 00:00:00 UTC  3800.14   100000000.0            26315
11  2017-08-13 00:00:00 UTC  4023.25   100000000.0            24855
12  2017-08-14 00:00:00 UTC  4175.70   100000000.0            23948
13  2017-08-15 00:00:00 UTC  4149.31   100000000

In [168]:
data = pd.read_csv(r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\updated_loan_data.csv")
fake = Faker()
def assign_random_names(df):
    df['Names'] = [fake.name() for _ in range(len(df))]
    cols = ['Names'] + [col for col in df.columns if col != 'Names']  # Reorder columns
    return df[cols]
    
data.columns = data.columns.str.strip().str.lower()  # Remove spaces & lowercase
print(data.columns)  

# Convert 'issue_month' to a full date format, assuming the 1st of each month
data['issue_date'] = pd.to_datetime(data['issue_month'], format='%y-%b') + pd.offsets.MonthBegin(0)

data = assign_random_names(data)
print(data.head(10))

Index(['names', 'profession', 'job_length', 'state', 'homeownership',
       'annual_income', 'debt_to_income', 'earliest_credit_line',
       'total_credit_lines', 'total_credit_limit', 'total_credit_utilized',
       'loan_purpose', 'loan_amount', 'term', 'interest_rate', 'installment',
       'total_payment', '10% of monthly', 'bitcoin strategy', 'issue_month',
       'issue_date'],
      dtype='object')
            Names             names          profession  job_length state  \
0      Stacy Frey       Tina Parker             realtor         7.0    CO   
1    Kelly Cooper       Joshua Park                 NaN         NaN    CA   
2     Sandra Kidd       Mark Keller                 NaN         NaN    CA   
3      Jason Hunt     Andrew Morgan                 NaN         NaN    CA   
4  Kristin Gordon       Diana Mills     quality control         1.0    MO   
5   Frank Beasley    Jennifer House                 NaN         NaN    NC   
6    James Turner     Michael Smith  payroll manag

In [169]:
total_loan_amount = data['loan_amount'].sum()
print(f"Total Loan Amount: ${total_loan_amount:,.2f}")



Total Loan Amount: $163,619,225.00


In [172]:

# finding which profession took the most loans
profession_loans = data.groupby('profession')['loan_amount'].sum().sort_values(ascending=False)

print(profession_loans.head(10))


profession
manager             3813425
owner               3557825
teacher             3256775
driver              1885700
sales               1695825
registered nurse    1542925
rn                  1428125
president           1317650
supervisor          1246625
truck driver        1053800
Name: loan_amount, dtype: int64


In [174]:
## fidning percentage of owners from the cali region

data['profession'] = data['profession'].str.lower()


total_owners = data[data['profession'] == 'owner'].shape[0]

# Number of "owners" in (CA)
owners_in_ca = data[(data['state'] == 'CA') & (data['profession'] == 'owner')].shape[0]

#percentage
percentage_owners_in_ca = (owners_in_ca / total_owners) * 100 if total_owners > 0 else 0

print(f"Percentage of 'owners' in CA: {percentage_owners_in_ca:.2f}%")


Percentage of 'owners' in CA: 16.18%


In [176]:
#total amount of loans given out this quater?

In [178]:
# percentage of each state loan amount relative to the total amount
state_loan_totals = data.groupby('state')['loan_amount'].sum()  # Use correct column name

# Calculate percentage of each state's total loan amount relative to the overall loan amount
state_loan_percentage = (state_loan_totals / state_loan_totals.sum()) * 100

# Convert to DataFrame properly
state_loan_percentage = state_loan_percentage.reset_index()

# Rename columns for clarity
state_loan_percentage.columns = ['state', 'loan_percentage']

#Sorting descending order to show the states with the highest loan amounts
state_loan_percentage = state_loan_percentage.sort_values('loan_percentage', ascending=False)

state_loan_percentage.index = range(1, len(state_loan_percentage) + 1)

top_10_states = state_loan_percentage.head(10)
print(top_10_states)






   state  loan_percentage
1     CA        13.490896
2     TX         8.406882
3     NY         7.694481
4     FL         6.847407
5     IL         4.141689
6     NJ         3.462827
7     GA         3.389149
8     OH         3.321202
9     NC         2.939279
10    PA         2.851636


In [198]:

## Convert 'issue_date' in loan_data to datetime format (same as bitcoin_date format)
# Check if the 'issue_date' is already timezone-aware
if data['issue_date'].dt.tz is None:
    # If not, localize to UTC
    data['issue_date'] = pd.to_datetime(data['issue_date']).dt.tz_localize('UTC')
else:
    # If already timezone-aware, convert to UTC
    data['issue_date'] = data['issue_date'].dt.tz_convert('UTC')

# Now, merge the two datasets on the date (nearest match for each customer)
merged_data = pd.merge_asof(data.sort_values('issue_date'), 
                            btc_data.sort_values('bitcoin_date'),
                            left_on='issue_date', right_on='bitcoin_date', 
                            direction='nearest')

# Now, calculate the satoshis bought for each customer using their 10% monthly repayment
merged_data['satoshis_bought'] = (merged_data['10% of monthly'] / merged_data['price']) * 100000000

# Show the first few rows of the merged data
print(merged_data[['Names', 'profession', 'issue_date', '10% of monthly', 'price', 'satoshis_bought']].head())



              Names                    profession                issue_date  \
0     Jordan Walton  program development director 2018-01-01 00:00:00+00:00   
1           Roy Ray                        sales  2018-01-01 00:00:00+00:00   
2       John Holmes                           NaN 2018-01-01 00:00:00+00:00   
3      Allison Bass                    supervisor 2018-01-01 00:00:00+00:00   
4  Michael Harrison                office manager 2018-01-01 00:00:00+00:00   

   10% of monthly     price  satoshis_bought  
0              52  14093.61    368961.536469  
1              48  14093.61    340579.879818  
2              11  14093.61     78049.555792  
3              61  14093.61    432820.263935  
4              18  14093.61    127717.454932  
